# Integrar Bases de Datos con Servicios externos

## Simulando una conexión a Bases de Datos remotas

Traer datos de servidores remotos gracias a **db_link**. Conectarnos a servidores externos dentro de una consulta.

Para este modulo crearemos una base de datos: **db-remota** que simule la conexion remota. Y una tabla para los usuarios vip:

- id: integer
- fecha: date

Y añadamos unos cuantos datos:

    INSERT INTO public.vip(
	id, fecha)
		VALUES (50, '2019-01-30'),
		(53, '2017-05-10'),
		(99, '2016-04-05');



### Conectandonos a la base remota

Desde la base de datos *transporte* nos conectaremso a la bd que acabamos de crear. Iremos ilustrando el paso a paso, con todos los problemas que se puedan presentar.

#### Funcion db_link sin instalar

Toda esta string de conexion, podriamos encapsularla en una vista o vista materializada, para no exponer ni el usuario ni el password.

![](https://i.imgur.com/LbJ4qsT.png)

Y dara un error *function dblink(unknown, unknown) does not exist*. Creamos la extension en nuestra BD:

    CREATE EXTENSION dblink

#### Error:a column definition list is required for functions returning "record"

Si volvemos a ejecutar, saldra ese error. Tendremso que llamarlo como una tabla local con el statemente **AS**, e indicarle en que formato los vamos a recibir. 

![](https://i.imgur.com/TkGSCzw.png)

Volvemos a ejecutar

#### Error: permission denied for table vip

El *usuario_consulta* no tiene permisos para conectarse, se los damos con la opcion *grant wizard*

![](https://i.imgur.com/e8eEKLp.png)

y ahora... Funciona:

![](https://i.imgur.com/aDZV1KR.png)

Script Final:

    SELECT * FROM 
    dblink('dbname=db-remota
	  port=5432
	  host=127.0.0.1
	  user=usuario_consulta
	  password=etc1234',
	  'SELECT id, fecha FROM vip')
	  AS datos_remotos (id integer, fecha date);

### Haciendo un Join

![](https://i.imgur.com/AtMXo2w.png)

Script:

    SELECT * FROM pasajero
	JOIN
	dblink('dbname=db-remota
		port=5432
		host=127.0.0.1
		user=usuario_consulta
		password=etc1234',
		'SELECT id, fecha FROM vip')
		AS datos_remotos (id integer, fecha date)
	ON (pasajero.id = datos_remotos.id);

#### Usando el statement USING

*USING* es un valor especial de las consultas, cuando los dos campos que vamos a comparar son iguales, en este caso en ambas tablas son *id*:

![](https://i.imgur.com/90U3EVb.png)

### Encapsulando los datos sensibles en una *VIEW*

Creamos una nueva *view* con este SQL de nombre **vip_view**

![](https://i.imgur.com/TkGSCzw.png)

Reescribimos y ejecutamos:

    SELECT * FROM pasajero
    JOIN
    vip_view
    USING(id);